In [ ]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import json
from types import SimpleNamespace
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchinfo import summary

%run data_preprocess.ipynb
%run helper.ipynb
%run wrapper.ipynb
%run models/ResNet_seq2seq.ipynb
%run visualize.ipynb
%run data_postprocess.ipynb

In [ ]:
dataport_appliance_data_661 = {
    "refrigerator": {
        "mean": 60.52,
        "std": 66.42,
        "window":100,
        'on_power_threshold': 50,
        'min': 0,
        'max': 683
    },
    "ev_car": {
        "mean": 204.14,
        "std": 737,
        'window':10,
        'on_power_threshold': 2000,
        'max': 3420,
        'min': 0,
    },
    "dishwasher": {
        "mean": 6.77,
        "std": 55.07,
        "window":3,
        'on_power_threshold': 10,
        'max': 960,
        'min': 0,
    },
    "clotheswasher": {
        "mean": 7.38,
        "std": 52.77,
        "window":100,
        'on_power_threshold': 20,
        'max': 1143,
        'min': 0,
    },
    "air": {
        "mean": 442.99,
        "std": 769.72,
        "window":100,
        'on_power_threshold': 20,
        'max': 2783,
        'min': 0,
    },
    "microwave": {
        "mean": 8.23,
        "std": 36.44,
        "window":100,
        'on_power_threshold': 200,
        'min': 0,
        'max': 980
    },
    "others": {
        "mean": 195,
        "std": 178,
        "window":100,
        'on_power_threshold': 0,
        'min': 0,
        'max': 1942
    }
}

# 8156
dataport_appliance_data_8156 = {
    "refrigerator": {
        "mean": 93.80,
        "std": 54.22,
        "window":100,
        'on_power_threshold': 50,
        'min': 6,
        'max': 562
    },
    "ev_car": {
        "mean": 207.76,
        "std": 750.25,
        'window':10,
        'on_power_threshold': 2000,
        'max': 3330,
        'min': 0,
    },
    "dishwasher": {
        "mean": 28.34,
        "std": 117.14,
        "window":3,
        'on_power_threshold': 10,
        'max': 936,
        'min': 0,
    },
    "clotheswasher": {
        "mean": 16.72,
        "std": 96.89,
        "window":100,
        'on_power_threshold': 20,
        'max': 1141,
        'min': 0,
    },
    "air": {
        "mean": 918.16,
        "std": 1214.59,
        "window":100,
        'on_power_threshold': 20,
        'max': 3949,
        'min': 0,
    },
    "microwave": {
        "mean": 13.89,
        "std": 73.69,
        "window":100,
        'on_power_threshold': 200,
        'min': 0,
        'max': 1678
    },
    "others": {
        "mean": 610.91,
        "std": 461.16,
        "window":100,
        'on_power_threshold': 0,
        'min': 76,
        'max': 5226
    }
}


In [ ]:
with open("Config/model_config.json") as json_data:
    data = json.load(json_data, object_hook=lambda d: SimpleNamespace(**d))
    model_params = data.resnet_seq2seq
    model_params.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
visualizer = Visualizer()
data_postprocess = DataPostProcess(model_params)

In [ ]:
helper = PrepareDataForQuantile(model_params)

In [ ]:
wrapper = Wrapper(model_params)

In [ ]:
model_evaluation = ModelEvaluation()

## EV

In [ ]:
#create another json with the ev car only
dataport_appliance_data_ev = {
    "ev_car": dataport_appliance_data_8156["ev_car"]
}

In [ ]:
dataport_appliance_data_ev

In [ ]:
train_loader_ev, test_loader_ev, val_loader_ev = helper.get_data_loaders(1, 2, model_params.lag_sizes[0])

In [ ]:
for x, y in train_loader_ev:
    print(x.shape, y.shape)
    break

In [ ]:
model_params.lag_size = model_params.lag_sizes[0]
model_ev = ResNetSeq2Seq(model_params).to(model_params.device)

In [ ]:
criterion = nn.MSELoss()  # Mean Squared Error loss
optimizer = optim.Adam(model_ev.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

In [ ]:
ground_truth_ev, prediction_ev = wrapper.regression_model_train(model_ev, train_loader_ev, val_loader_ev, optimizer, criterion, scheduler)

In [ ]:
visualizer.visualize_plot( ground_truth_ev.cpu().detach().numpy(), prediction_ev.cpu().detach().numpy(), 00, 186000, 0)

In [ ]:
ground_truth_test_ev, prediction_test_ev = wrapper.regression_model_test(model_ev, test_loader_ev)

In [ ]:
#create another json with the ev car only
dataport_appliance_data_ev_test = {
    "ev_car": dataport_appliance_data_661["ev_car"]
}

In [ ]:
ground_truth_ev_scaled, prediction_ev_scaled = data_postprocess.reverse_scale_data_unetnilm(ground_truth_test_ev.cpu().detach().numpy(), prediction_test_ev.cpu().detach().numpy(), dataport_appliance_data_ev_test, 'standard')

In [ ]:
ground_truth_ev_scaled = ground_truth_test_ev.cpu().detach().numpy()
prediction_ev_scaled = prediction_test_ev.cpu().detach().numpy()

In [ ]:
visualizer.visualize_plot( ground_truth_ev_scaled, prediction_ev_scaled, 00, 180000, 0)

In [ ]:
mae, eac, nde = wrapper.evaluation_metrics(ground_truth_ev_scaled[:, 0], prediction_ev_scaled[:, 0])

In [ ]:
print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)

In [ ]:
dataframe = pd.DataFrame({
   "ground_truth": ground_truth_ev_scaled[:, 0],
   "prediction": prediction_ev_scaled[:, 0]
})

dataframe.to_csv("Results/output/resnet_8156_seq2seq_ev_withev.csv", index=False)

## Washing machine

In [ ]:
#write the whole code for the washing maching
dataport_appliance_data_wm = {
    "clotheswasher": dataport_appliance_data_8156["clotheswasher"]
}

train_loader_wm, test_loader_wm, val_loader_wm = helper.get_data_loaders(3, 4, model_params.lag_sizes[1])

for x, y in train_loader_wm:
    print(x.shape, y.shape)
    break

model_params.lag_size = model_params.lag_sizes[1]

model_wm = ResNetSeq2Seq(model_params).to(model_params.device)

criterion = nn.MSELoss()  # Mean Squared Error loss
optimizer = optim.Adam(model_wm.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

summary(model_wm, input_size=x.shape)

In [ ]:
ground_truth_wm, prediction_wm = wrapper.regression_model_train(model_wm, train_loader_wm, val_loader_wm, optimizer, criterion, scheduler)

In [ ]:
visualizer.visualize_plot( ground_truth_wm.cpu().detach().numpy(), prediction_wm.cpu().detach().numpy(), 0, 7000, 0)

In [ ]:
ground_truth_test_wm, prediction_test_wm = wrapper.regression_model_test(model_wm, test_loader_wm)

In [ ]:
ground_truth_wm_scaled, prediction_wm_scaled = data_postprocess.reverse_scale_data_unetnilm(ground_truth_test_wm.cpu().detach().numpy(), prediction_test_wm.cpu().detach().numpy(), dataport_appliance_data_wm, 'standard')

In [ ]:
ground_truth_wm_scaled = ground_truth_test_wm.cpu().detach().numpy()
prediction_wm_scaled = prediction_test_wm.cpu().detach().numpy()

In [ ]:
visualizer.visualize_plot( ground_truth_wm_scaled, prediction_wm_scaled, 00, 180000, 0)

In [ ]:
mae, eac, nde = wrapper.evaluation_metrics(ground_truth_wm_scaled[:, 0], prediction_wm_scaled[:, 0])

In [ ]:
print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)

In [ ]:
data_frame = pd.DataFrame({
   "ground_truth": ground_truth_wm_scaled[:, 0],
    "prediction": prediction_wm_scaled[:, 0]
})

data_frame.to_csv("Results/output/resnet_8156_seq2seq_wm_withev.csv", index=False)

## Refrigerator

In [ ]:
##write the whole code for the refrigerator
dataport_appliance_data_ref = {
    "refrigerator": dataport_appliance_data["refrigerator"]
}

train_loader_ref, test_loader_ref, val_loader_ref = helper.get_data_loaders(0, 1, model_params.lag_sizes[2])

for x, y in train_loader_ref:
    print(x.shape, y.shape)
    break

model_params.lag_size = model_params.lag_sizes[2]

model_ref = ResNetSeq2Seq(model_params).to(model_params.device)

criterion = nn.MSELoss()  # Mean Squared Error loss
optimizer = optim.Adam(model_ref.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

ground_truth_ref, prediction_ref = wrapper.regression_model_train(model_ref, train_loader_ref, val_loader_ref, optimizer, criterion, scheduler)

In [ ]:
visualizer.visualize_plot( ground_truth_ref.cpu().detach().numpy(), prediction_ref.cpu().detach().numpy(), 0, 7000, 0)

In [ ]:
ground_truth_test_ref, prediction_test_ref = wrapper.regression_model_test(model_ref, test_loader_ref)

In [ ]:
ground_truth_ref_scaled, prediction_ref_scaled = data_postprocess.reverse_scale_data_unetnilm(ground_truth_test_ref.cpu().detach().numpy(), prediction_test_ref.cpu().detach().numpy(), dataport_appliance_data_ref, 'standard')

In [ ]:

visualizer.visualize_plot( ground_truth_ref_scaled, prediction_ref_scaled, 00, 500000, 0)

In [ ]:
#negative values to zero
prediction_ref_scaled[prediction_ref_scaled < 0] = 0
ground_truth_ref_scaled[ground_truth_ref_scaled < 0] = 0

In [ ]:
mae, eac, nde = wrapper.evaluation_metrics(ground_truth_ref_scaled[:, 0], prediction_ref_scaled[:, 0])

In [ ]:

print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)

In [ ]:
dataframe = pd.DataFrame({
   "ground_truth": ground_truth_ref_scaled[:, 0],
    "prediction": prediction_ref_scaled[:, 0]
})

dataframe.to_csv("Results/output/resnet_8156_seq2seq_ref_withev.csv", index=False)

## Microwave

In [ ]:
## write the whole code for the microwave
dataport_appliance_data_mw = {
    "microwave": dataport_appliance_data["microwave"]
}

train_loader_mw, test_loader_mw, val_loader_mw = helper.get_data_loaders(2, 3, model_params.lag_sizes[3])

for x, y in train_loader_mw:
    print(x.shape, y.shape)
    break

model_params.lag_size = model_params.lag_sizes[3]

model_mw = ResNetSeq2Seq(model_params).to(model_params.device)

criterion = nn.MSELoss()  # Mean Squared Error loss
optimizer = optim.Adam(model_mw.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

ground_truth_mw, prediction_mw = wrapper.regression_model_train(model_mw, train_loader_mw, val_loader_mw, optimizer, criterion, scheduler)

In [ ]:

visualizer.visualize_plot( ground_truth_mw.cpu().detach().numpy(), prediction_mw.cpu().detach().numpy(), 0, 7000, 0)

In [ ]:
ground_truth_test_mw, prediction_test_mw = wrapper.regression_model_test(model_mw, test_loader_mw)

In [ ]:
ground_truth_mw_scaled, prediction_mw_scaled = data_postprocess.reverse_scale_data_unetnilm(ground_truth_test_mw.cpu().detach().numpy(), prediction_test_mw.cpu().detach().numpy(), dataport_appliance_data_mw, 'standard')

In [ ]:
visualizer.visualize_plot( ground_truth_mw_scaled, prediction_mw_scaled, 00, 18000, 0)

In [ ]:
mae, eac, nde = wrapper.evaluation_metrics(ground_truth_mw_scaled[:, 0], prediction_mw_scaled[:, 0])

print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)

In [ ]:
mae, eac, nde = wrapper.evaluation_metrics(ground_truth_mw_scaled[:, 0], prediction_mw_scaled[:, 0])

print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)

In [ ]:
dataframe_mx = pd.DataFrame({
   "ground_truth": ground_truth_mw_scaled[:, 0],
    "prediction": prediction_mw_scaled[:, 0]
})
dataframe_mx.to_csv("Results/output/resnet_8156_seq2seq_mw_withoutv.csv", index=False)

## air


In [ ]:
## write the whole code for the air conditioner
dataport_appliance_data_air = {
    "air": dataport_appliance_data["air"]
}
train_loader_air, test_loader_air, val_loader_air = helper.get_data_loaders(3, 4, model_params.lag_sizes[4])
for x, y in train_loader_air:
    print(x.shape, y.shape)
    break

model_params.lag_size = model_params.lag_sizes[4]
model_air = ResNetSeq2Seq(model_params).to(model_params.device)
criterion = nn.MSELoss()  # Mean Squared Error loss
optimizer = optim.Adam(model_air.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

ground_truth_air, prediction_air = wrapper.regression_model_train(model_air, train_loader_air, val_loader_air, optimizer, criterion, scheduler)

In [ ]:
visualizer.visualize_plot( ground_truth_air.cpu().detach().numpy(), prediction_air.cpu().detach().numpy(), 0, 7000, 0)

In [ ]:
ground_truth_test_air, prediction_test_air = wrapper.regression_model_test(model_air, test_loader_air)

In [ ]:
ground_truth_air_scaled, prediction_air_scaled = data_postprocess.reverse_scale_data_unetnilm(ground_truth_test_air.cpu().detach().numpy(), prediction_test_air.cpu().detach().numpy(), dataport_appliance_data_air, 'standard')

In [ ]:
visualizer.visualize_plot( ground_truth_air_scaled, prediction_air_scaled, 00, 180000, 0)

In [ ]:
mae, eac, nde = wrapper.evaluation_metrics(ground_truth_air_scaled[:, 0], prediction_air_scaled[:, 0])
print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)

In [ ]:
dataframe = pd.DataFrame({
   "ground_truth": ground_truth_air_scaled[:, 0],
    "prediction": prediction_air_scaled[:, 0]
})
dataframe.to_csv("Results/output/resnet_8156_seq2seq_air_withoutev.csv", index=False)

## Dishwasher

In [ ]:
# write the whole code for the dishwasher
dataport_appliance_data_dishwasher = {
    "dishwasher": dataport_appliance_data["dishwasher"]
}

train_loader_dishwasher, test_loader_dishwasher, val_loader_dishwasher = helper.get_data_loaders(1, 2, model_params.lag_sizes[5])
for x, y in train_loader_dishwasher:
    print(x.shape, y.shape)
    break

model_params.lag_size = model_params.lag_sizes[5]
model_dishwasher = ResNetSeq2Seq(model_params).to(model_params.device)
criterion = nn.MSELoss()  # Mean Squared Error loss
optimizer = optim.Adam(model_dishwasher.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

ground_truth_dishwasher, prediction_dishwasher = wrapper.regression_model_train(model_dishwasher, train_loader_dishwasher, val_loader_dishwasher, optimizer, criterion, scheduler)

In [ ]:

visualizer.visualize_plot( ground_truth_dishwasher.cpu().detach().numpy(), prediction_dishwasher.cpu().detach().numpy(), 0, 7000, 0)

In [ ]:

ground_truth_test_dishwasher, prediction_test_dishwasher = wrapper.regression_model_test(model_dishwasher, test_loader_dishwasher)

In [ ]:
ground_truth_dishwasher_scaled, prediction_dishwasher_scaled = data_postprocess.reverse_scale_data_unetnilm(ground_truth_test_dishwasher.cpu().detach().numpy(), prediction_test_dishwasher.cpu().detach().numpy(), dataport_appliance_data_dishwasher, 'standard')

In [ ]:
visualizer.visualize_plot( ground_truth_dishwasher_scaled, prediction_dishwasher_scaled, 00, 180000, 0)

In [ ]:
mae, eac, nde = wrapper.evaluation_metrics(ground_truth_dishwasher_scaled[:, 0], prediction_dishwasher_scaled[:, 0])
print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)

In [ ]:
dataframe_dishwasher = pd.DataFrame({
   "ground_truth": ground_truth_dishwasher_scaled[:, 0],
    "prediction": prediction_dishwasher_scaled[:, 0]
})
dataframe_dishwasher.to_csv("Results/output/resnet_8156_seq2seq_dishwasher_withoutev.csv", index=False)

## Others

In [ ]:
# write the whole code for the others
dataport_appliance_data_others = {
    "others": dataport_appliance_data["others"]
}

train_loader_others, test_loader_others, val_loader_others = helper.get_data_loaders(5, 6, model_params.lag_sizes[6])
for x, y in train_loader_others:
    print(x.shape, y.shape)
    break

model_params.lag_size = model_params.lag_sizes[6]
model_others = ResNetSeq2Seq(model_params).to(model_params.device)
criterion = nn.MSELoss()  # Mean Squared Error loss
optimizer = optim.Adam(model_others.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
ground_truth_others, prediction_others = wrapper.regression_model_train(model_others, train_loader_others, val_loader_others, optimizer, criterion, scheduler)

In [ ]:

visualizer.visualize_plot( ground_truth_others.cpu().detach().numpy(), prediction_others.cpu().detach().numpy(), 0, 7000, 0)


In [ ]:
ground_truth_test_others, prediction_test_others = wrapper.regression_model_test(model_others, test_loader_others)

In [ ]:
ground_truth_others_scaled, prediction_others_scaled = data_postprocess.reverse_scale_data_unetnilm(ground_truth_test_others.cpu().detach().numpy(), prediction_test_others.cpu().detach().numpy(), dataport_appliance_data_others, 'standard')

In [ ]:
visualizer.visualize_plot( ground_truth_others_scaled, prediction_others_scaled, 00, 180000, 0)

In [ ]:
mae, eac, nde = wrapper.evaluation_metrics(ground_truth_others_scaled[:, 0], prediction_others_scaled[:, 0])
print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)

In [ ]:
dataframe_others = pd.DataFrame({
   "ground_truth": ground_truth_others_scaled[:, 0],
    "prediction": prediction_others_scaled[:, 0]
})
dataframe_others.to_csv("Results/output/resnet_8156_seq2seq_others_withev.csv", index=False)